# Linear Layer

In [1]:
import torch

## Raw Linear Layer

$$\begin{gathered}
y=x\cdot{W}+b, \\
\text{where }x\in\mathbb{R}^{N\times{n}}\text{, }y\in\mathbb{R}^{N\times{m}}. \\
\\
\text{Thus, }W\in\mathbb{R}^{n\times{m}}\text{ and }b\in\mathbb{R}^m.
\end{gathered}$$

W = 가중치

위 수식을 구현해보기

In [2]:
W = torch.FloatTensor([[1, 2],
                       [3, 4],
                       [5, 6]])

# W = ([n,m]) = ([행,렬]) = ([3,2])
b = torch.FloatTensor([2, 2])
# b = ([m]) = (벡터?) = ([2])

In [3]:
print(W.size())
print(b.size())

torch.Size([3, 2])
torch.Size([2])


In [4]:
def linear(x, W, b):
    y = torch.matmul(x, W) + b
    
# x = ([N,n])
# W = ([n,m])
# x * W = ([N,n] * [n,m]) = ([N, m])
#현재 n = 3, m = 2, N은 모름
    
    return y
# y = ([N,m])

In [5]:
x = torch.FloatTensor([[1, 1, 1],
                       [2, 2, 2],
                       [3, 3, 3],
                       [4, 4, 4]])
#이제 X를 실제로 선언

print(x.size())
#대문자 N은 4

torch.Size([4, 3])


In [6]:
y = linear(x, W, b) #이제 y를 구할수있음.

In [7]:
print(y.size())

torch.Size([4, 2])


## nn.Module

#### PyTorch 모델의 기본 구조

1. PyTorch 내장 모델 뿐만 아니라 사용자 정의 모델도 반드시 이 구조를 따라야 한다.

import torch.nn as nn
import torch.nn.functional as F

class Model_Name(nn.Module):
    def __init__(self):
    
        super(Model_Name, self).__init__()
        self.module1 = ...
        self.module2 = ...
        
        """
        ex)
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)
        """

    def forward(self, x):
    
        x = some_function1(x)
        x = some_function2(x)
        
        """
        ex)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        """
        return x
        
model = Model_Name() # 여기에 변수를 넣어주면 됨.

2. nn.Module

__init__(self): initialize; 내가 사용하고 싶은, 내 신경망 모델에 사용될 구성품들을 정의 및 초기화 하는 메소드이다. 
forward(self, x): specify the connections;  이닛에서 정의된 구성품들을 연결하는 메소드이다. 


### 틀린코드

In [8]:
import torch.nn as nn

In [9]:
class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2): # init함수에서는 내가 나중에 forward에서 쓸 함수를 미리 선언하는것
        self.input_dim = input_dim #input dimension = n
        self.output_dim = output_dim # output dimension이 = m
        
        super().__init__()
        # W와 b를 생성한다.
        self.W = torch.FloatTensor(input_dim, output_dim) #([3,2])로 생성
        self.b = torch.FloatTensor(output_dim) #([2])라는 차원의 벡터로 생성

    # You should override 'forward' method to implement detail.
    # The input arguments and outputs can be designed as you wish.
    
    def forward(self, x): #forward는 행동을 선언한다
        # |x| = (batch_size(병렬로 처리할 샘플의 숫자), input_dim(입력벡터의 크기))
        y = torch.matmul(x, self.W) + self.b
        # |y| = (batch_size, input_dim) * (input_dim, output_dim)
        #     = (batch_size, output_dim)
        
        return y
    

### nn.Linear

* torch.nn.Linear(input_size, output_size)
* 들어오는 float32형 input 데이터에 대해 y=wx+b 형태의 선형 변환을 수행하는 메소드인 것을 알 수 있다.
* Linear를 취할 때 정수형은 float으로 만들어 소수점 .0을 달아줘야한다 

In [10]:
linear = MyLinear(3, 2)

y = linear(x) # x통과시키면 y가 나옴

In [11]:
print(y.size())

torch.Size([4, 2])


In [12]:
for p in linear.parameters():
    print(p)
    

이 코드들 수행안됨. 잘못된 코드

You can see that there is no weight parameters to learn.
Above way can forward(or calculate) values, but it cannot be trained.

### 올바른 코드 Correct way: nn.Parameter

In [13]:
class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        super().__init__()
        
        self.W = nn.Parameter(torch.FloatTensor(input_dim, output_dim)) #nn.Parameter 를 써야함
        self.b = nn.Parameter(torch.FloatTensor(output_dim))
        
    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = torch.matmul(x, self.W) + self.b
        # |y| = (batch_size, input_dim) * (input_dim, output_dim)
        #     = (batch_size, output_dim)
        
        return y

Reference: https://pytorch.org/docs/stable/nn.html#torch.nn.Parameter

A kind of Tensor that is to be considered a module parameter.

Parameters are Tensor subclasses, that have a very special property when used with Module s - when they’re assigned as Module attributes they are automatically added to the list of its parameters, and will appear e.g. in parameters() iterator. Assigning a Tensor doesn’t have such effect. This is because one might want to cache some temporary state, like last hidden state of the RNN, in the model. If there was no such class as Parameter, these temporaries would get registered too.

In [14]:
linear = MyLinear(3, 2)

y = linear(x)

In [15]:
print(y.size())

torch.Size([4, 2])


In [16]:
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[ 0.0000e+00, -0.0000e+00],
        [ 9.1002e+31, -2.8586e-42],
        [ 8.4078e-45,  0.0000e+00]], requires_grad=True)
Parameter containing:
tensor([4.7428e+30, 7.1429e+31], requires_grad=True)


## nn.Linear

In [17]:
linear = nn.Linear(3, 2)

y = linear(x)

In [18]:
print(y.size())

torch.Size([4, 2])


#### parameters = weight, bias

In [19]:
for p in linear.parameters(): # parameters() 메소드는 모듈의 파라메터들을 iterator로 반환함
    print(p)

Parameter containing:
tensor([[-0.3386, -0.5355, -0.4991],
        [ 0.1993,  0.4776,  0.1894]], requires_grad=True)
Parameter containing:
tensor([0.1819, 0.0941], requires_grad=True)


### nn.Module can contain other nn.Module's child classes.

In [20]:
class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        super().__init__()
        
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = self.linear(x)
        # |y| = (batch_size, output_dim)
        
        return y

In [21]:
linear = MyLinear(3, 2)

y = linear(x)

In [22]:
print(y.size())

torch.Size([4, 2])


In [23]:
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[-0.1267,  0.0563,  0.3951],
        [ 0.2291,  0.3214,  0.2595]], requires_grad=True)
Parameter containing:
tensor([0.3659, 0.4013], requires_grad=True)
